In [1]:
#%pip install transformers
#%pip install torch
#%pip install datasets

In [5]:
#finetuning on IMDb
from datasets import load_dataset

imdb_dataset = load_dataset("imdb")
imdb_dataset

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [6]:
imdb_dataset["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [7]:
sample = imdb_dataset["unsupervised"].shuffle(seed=97).select(range(3))

In [8]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = imdb_dataset.map(
    tokenize_function, batched=True, remove_columns=["text", "label"]
)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 50000
    })
})

In [9]:
#get max context leghth for chucking
tokenizer.model_max_length

512

In [10]:
chunk_size = 256
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) if isinstance(examples[k][0], list) else examples[k] for k in examples.keys()}
    # Compute the total length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # Drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of chunk_size
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new "labels" column that is a copy of "input_ids"
    result["labels"] = result["input_ids"].copy()
    return result


In [11]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 30639
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 29946
    })
    unsupervised: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 61465
    })
})

In [12]:
#deviate from the tutoral at this point, and look at documention instead 

In [13]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

tokenizer.pad_token = "[PAD]" #tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [14]:

training_args = TrainingArguments(
    output_dir="my_awesome_eli5_mlm_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=2,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

/var/folders/06/f0g8dl3x5414djvzzgmvfz080000gn/T/ipykernel_30257/1218978447.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  7%|▋         | 500/7660 [01:49<25:56,  4.60it/s]

{'loss': 2.572, 'grad_norm': 7.295171737670898, 'learning_rate': 1.8694516971279374e-05, 'epoch': 0.13}


 13%|█▎        | 1000/7660 [04:14<24:47,  4.48it/s]  

{'loss': 2.5044, 'grad_norm': 6.966341018676758, 'learning_rate': 1.7389033942558747e-05, 'epoch': 0.26}


 20%|█▉        | 1500/7660 [06:06<22:55,  4.48it/s]

...

 98%|█████████▊| 7500/7660 [32:20<00:35,  4.56it/s]

{'loss': 2.3032, 'grad_norm': 6.8098249435424805, 'learning_rate': 4.1775456919060057e-07, 'epoch': 1.96}


100%|██████████| 7660/7660 [37:14<00:00,  3.43it/s]

{'eval_loss': 2.2162673473358154, 'eval_runtime': 257.1096, 'eval_samples_per_second': 116.472, 'eval_steps_per_second': 14.562, 'epoch': 2.0}
{'train_runtime': 2234.6628, 'train_samples_per_second': 27.422, 'train_steps_per_second': 3.428, 'train_loss': 2.3837176489767766, 'epoch': 2.0}


TrainOutput(global_step=7660, training_loss=2.3837176489767766, metrics={'train_runtime': 2234.6628, 'train_samples_per_second': 27.422, 'train_steps_per_second': 3.428, 'total_flos': 4061541261625344.0, 'train_loss': 2.3837176489767766, 'epoch': 2.0})

In [17]:
output_dir = "finetuned_models"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('finetuned_models/tokenizer_config.json',
 'finetuned_models/special_tokens_map.json',
 'finetuned_models/vocab.txt',
 'finetuned_models/added_tokens.json',
 'finetuned_models/tokenizer.json')

In [18]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

# Replace AutoModelForMaskedLM with the correct class for your task, e.g., AutoModelForSequenceClassification
model = AutoModelForMaskedLM.from_pretrained("finetuned_models/")
tokenizer = AutoTokenizer.from_pretrained("finetuned_models/")


In [24]:
from transformers import pipeline

mask_filler_base = pipeline(
    "fill-mask", model=model_checkpoint, device="mps"
)

mask_filler_finetuned = pipeline(
    "fill-mask", model="finetuned_models/", device="mps"
)

In [29]:
text = "This is a great [MASK]"

#origional model
for pred in mask_filler_base(text):
    print(f"Origional >>> {pred['sequence']}")

print("\n")

#new IMDb finetuned model 
for pred in mask_filler_finetuned(text):
    print(f"Updated model >>> {pred['sequence']}")


Origional >>> this is a great!
Origional >>> this is a great.
Origional >>> this is a great deal
Origional >>> this is a great adventure
Origional >>> this is a great ;


Updated model >>> this is a great!
Updated model >>> this is a great.
Updated model >>> this is a great film
Updated model >>> this is a great movie
Updated model >>> this is a great idea
